In [21]:
import matplotlib.pyplot as plt
vega_path = './Star Images/Vega.jpg'
other_path = './Star Images/Alnitak.jpg'

In [19]:
from PIL import Image
import numpy as np
from skimage import segmentation, color, filters

def calculate_luminosity(image_path):
    # Load the image
    img = Image.open(image_path)
    
    # Convert the image to a NumPy array for easier manipulation
    img_array = np.array(img)
    
    # Convert the image to grayscale if it is color
    if img_array.ndim == 3:
        gray_img = color.rgb2gray(img_array)
    else:
        gray_img = img_array
    
    # Apply Gaussian blur to the grayscale image
    blurred_img = filters.gaussian(gray_img, sigma=2)
    
    # Use adaptive thresholding to separate star and background
    threshold = filters.threshold_local(blurred_img, block_size=51, offset=0.1)
    star_mask = (blurred_img > threshold).astype(np.uint8)
    
    # Invert the mask to get the background
    background_mask = 1 - star_mask
    
    # Apply the masks to the original image
    star_roi = img_array * np.expand_dims(star_mask, axis=-1)
    background_roi = img_array * np.expand_dims(background_mask, axis=-1)
    
    # Calculate average pixel brightness for the star and background
    star_brightness = np.mean(star_roi)
    background_brightness = np.mean(background_roi)
    
    # Calculate luminosity by subtracting background brightness from star brightness
    luminosity = star_brightness - background_brightness
    
    return luminosity

# Replace 'your_star_image.jpg' and 'your_vega_image.jpg' with your actual file paths
star_luminosity = calculate_luminosity(other_path)
vega_luminosity = calculate_luminosity(vega_path)

# Calculate the ratio of luminosities
luminosity_ratio = - star_luminosity / vega_luminosity

print(f"Luminosity of Star: {star_luminosity}")
print(f"Luminosity of Vega: {vega_luminosity}")
print(f"Luminosity Ratio (Star / Vega): {luminosity_ratio}")


Luminosity of Star: 71.76946533333333
Luminosity of Vega: 88.35669466666667
Luminosity Ratio (Star / Vega): -0.8122696939274369


In [70]:
import cv2
import numpy as np

def extract_star(image_path):
    # Read the image
    img = cv2.imread(image_path)

    # Convert the image to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Apply thresholding to separate the star from the background
    _, thresholded = cv2.threshold(gray, 220, 255, cv2.THRESH_BINARY)

    # Find contours in the thresholded image
    contours, _ = cv2.findContours(thresholded, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Create a mask to keep only the star
    mask = np.zeros_like(img)
    cv2.drawContours(mask, contours, -1, (255, 255, 255), thickness=cv2.FILLED)

    # Apply the mask to the original image to get the star
    star = cv2.bitwise_and(img, img, mask=mask)

    # Invert the mask to get the removed background
    background_mask = cv2.bitwise_not(mask)

    # Create a white background of the same size as the original image
    white_background = np.ones_like(img, dtype=np.uint8) * 255

    # Apply the inverted mask to the white background to get the background
    background = cv2.bitwise_and(white_background, white_background, mask=background_mask)

    return star, background

# Replace 'your_image_path.jpg' with the actual path to your star image
output_image, bg = extract_star("./Star Images/Unurgunite.jpg")

# Display the result
# plt.imshow(output_image)
plt.imshow(bg)
plt.show()

error: OpenCV(4.9.0) /Users/xperience/GHA-OpenCV-Python2/_work/opencv-python/opencv-python/opencv/modules/core/src/arithm.cpp:230: error: (-215:Assertion failed) (mtype == CV_8U || mtype == CV_8S) && _mask.sameSize(*psrc1) in function 'binary_op'


In [5]:
from rembg import remove
import glob

input_path = './Star Images/*.jpg'
inputs = glob.glob(input_path)

for i in inputs:
    output = remove(open(i, 'rb').read())
    with open("./Star Images Removed Backgrounds/" + i.split("/")[-1], 'wb') as f:
        f.write(output)